In [1]:
# === Cell 1: 2D Heat Equation Simulation and Save to CSV ===

import numpy as np
import pandas as pd

# Parameters
nx, ny = 50, 50
nt = 100
lx, ly = 1.0, 1.0
dx, dy = lx/nx, ly/ny
dt = 0.001
alpha = 0.01  # thermal diffusivity

x = np.linspace(0, lx, nx)
y = np.linspace(0, ly, ny)
t = np.arange(0, nt*dt, dt)

# Initialize temperature field
u = np.zeros((nx, ny, nt))
X, Y = np.meshgrid(x, y, indexing="ij")
u[:,:,0] = np.exp(-50*((X-0.5)**2 + (Y-0.5)**2))  # Gaussian IC

# Time stepping (explicit FTCS)
for k in range(0, nt-1):
    u_xx = (np.roll(u[:,:,k], -1, axis=0) - 2*u[:,:,k] + np.roll(u[:,:,k], 1, axis=0)) / dx**2
    u_yy = (np.roll(u[:,:,k], -1, axis=1) - 2*u[:,:,k] + np.roll(u[:,:,k], 1, axis=1)) / dy**2
    u[:,:,k+1] = u[:,:,k] + alpha * dt * (u_xx + u_yy)

# Save to CSV
X, Y, T = np.meshgrid(x, y, t, indexing="ij")
df = pd.DataFrame({
    "x": X.ravel(),
    "y": Y.ravel(),
    "t": T.ravel(),
    "u": u.ravel()
})
df.to_csv("rawdata.csv", index=False)
print("Saved rawdata.csv with shape:", df.shape)


Saved rawdata.csv with shape: (250000, 4)


In [2]:
# === Cell 2: Load Data ===
df = pd.read_csv("rawdata.csv")
print(df.head())

x = np.unique(df["x"].values)
y = np.unique(df["y"].values)
t = np.unique(df["t"].values)

u = df["u"].values.reshape(len(x), len(y), len(t))
print("u.shape =", u.shape)


     x    y      t             u
0  0.0  0.0  0.000  1.388794e-11
1  0.0  0.0  0.001  1.508033e-11
2  0.0  0.0  0.002  1.634005e-11
3  0.0  0.0  0.003  1.766995e-11
4  0.0  0.0  0.004  1.907300e-11
u.shape = (50, 50, 100)


In [3]:
# === Cell 3: Derivatives (central differences, higher order) ===

nt, nx, ny = len(t), len(x), len(y)

u_t   = np.zeros_like(u)
u_2t  = np.zeros_like(u)
u_x   = np.zeros_like(u)
u_2x  = np.zeros_like(u)
u_3x  = np.zeros_like(u)
u_y   = np.zeros_like(u)
u_2y  = np.zeros_like(u)
u_3y  = np.zeros_like(u)

for k in range(1, nt-1):
    for i in range(2, nx-2):
        for j in range(2, ny-2):
            # Time derivatives
            u_t[i,j,k]  = (u[i,j,k+1] - u[i,j,k-1]) / (2*dt)
            u_2t[i,j,k] = (u[i,j,k+1] - 2*u[i,j,k] + u[i,j,k-1]) / (dt**2)

            # Space derivatives (x)
            u_x[i,j,k]  = (u[i+1,j,k] - u[i-1,j,k]) / (2*dx)
            u_2x[i,j,k] = (u[i+1,j,k] - 2*u[i,j,k] + u[i-1,j,k]) / (dx**2)
            u_3x[i,j,k] = (u[i+2,j,k] - 2*u[i+1,j,k] + 2*u[i-1,j,k] - u[i-2,j,k]) / (2*dx**3)

            # Space derivatives (y)
            u_y[i,j,k]  = (u[i,j+1,k] - u[i,j-1,k]) / (2*dy)
            u_2y[i,j,k] = (u[i,j+1,k] - 2*u[i,j,k] + u[i,j-1,k]) / (dy**2)
            u_3y[i,j,k] = (u[i,j+2,k] - 2*u[i,j+1,k] + 2*u[i,j-1,k] - u[i,j-2,k]) / (2*dy**3)

print("Derivatives computed: u_t, u_2t, u_x, u_2x, u_3x, u_y, u_2y, u_3y")


Derivatives computed: u_t, u_2t, u_x, u_2x, u_3x, u_y, u_2y, u_3y


In [4]:
# Flatten features
Ut   = u_t.ravel()
U    = u.ravel()
Ux   = u_x.ravel()
U2x  = u_2x.ravel()
U3x  = u_3x.ravel()
Uy   = u_y.ravel()
U2y  = u_2y.ravel()
U3y  = u_3y.ravel()

X_features = np.column_stack([U, Ux, U2x, U3x, Uy, U2y, U3y])
feature_names = ["u", "u_x", "u_xx", "u_xxx", "u_y", "u_yy", "u_yyy"]

print("X_features shape:", X_features.shape)
print("Ut shape:", Ut.shape)


X_features shape: (250000, 7)
Ut shape: (250000,)


In [5]:
# === Cell 5: Run PySR symbolic regression ===
# !pip install pysr

from pysr import PySRRegressor

model = PySRRegressor(
    niterations=40,
    unary_operators=[],
    binary_operators=["+", "-", "*", "/"],
    populations=20,
    model_selection="best",
    loss="loss(x, y) = (x - y)^2",
    verbosity=1,
)

model.fit(X_features, Ut)



Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


c:\Users\adil\anaconda3\envs\ham\Lib\site-packages\pysr\sr.py:1036: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
c:\Users\adil\anaconda3\envs\ham\Lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\adil\anaconda3\envs\ham\Lib\site-packages\pysr\sr.py:2265: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://ai.damtp.cam.ac.uk/pysr/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
Compiling Julia backend...
[ Info: Note: you are running with more than 10,000 datapoints


Expressions evaluated per second: 0.000e+00
Progress: 0 / 800 total iterations (0.000%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════════════════════════════════════════════
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 8.040e+02
Progress: 29 / 800 total iterations (3.625%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           4.813e-02  0.000e+00  y = -1.2851e-05
3           2.104e-02  4.138e-

[ Info: Final population:
[ Info: Results saved to:


,model_selection,'best'
,binary_operators,"['+', '-', ...]"
,unary_operators,[]
,expression_spec,None
,niterations,40
,populations,20
,population_size,27
,max_evals,None
,maxsize,30
,maxdepth,None
,warmup_maxsize_by,None


In [13]:
print(model.equations_)


    complexity          loss  \
0            1  4.812957e-02   
1            3  1.599622e-02   
2            5  3.221232e-08   
3            7  3.221143e-08   
4            9  3.221074e-08   
5           11  2.190079e-08   
6           13  1.499120e-08   
7           15  1.240441e-08   
8           19  1.237127e-08   
9           21  7.607031e-09   
10          23  7.170517e-09   
11          25  6.906248e-09   
12          27  5.978815e-09   

                                             equation     score  \
0                                        1.2779228e-5  0.000000   
1                                    x5 * 0.013371806  0.550772   
2                             (x5 + x2) * 0.010014186  6.557757   
3            ((x5 + x2) - -3.628745e-5) * 0.010014187  0.000014   
4   (((x2 + x5) - -3.628745e-5) - -3.628745e-5) * ...  0.000011   
5   (x5 * 0.010018738) + (((x0 * -0.0857366) - x2)...  0.192889   
6   ((x4 * 3.796528e-5) * x4) + (((x5 * -0.9989769...  0.189530   
7   ((((x5 * -0